In [ ]:
import sys
repo_path = '/home/micael.verissimo/paper_lzt/exp-connection-hep/'
sys.path.insert(0, repo_path)

In [2]:
import os

import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from src.callbacks import sp_index
from src.decorators import Summary
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

2025-04-26 01:38:57.107046: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-26 01:38:57.163402: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-26 01:38:57.163449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-26 01:38:57.165524: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-26 01:38:57.176123: I tensorflow/core/platform/cpu_feature_guar

Welcome to JupyROOT 6.30/02


In [3]:
prefix = "cl_ring_%i"

# rings presample 
presample = [prefix % iring for iring in range(8 // 2)]

# EM1 list
sum_rings = 8
em1 = [prefix % iring for iring in range(sum_rings, sum_rings + (64 // 2))]

# EM2 list
sum_rings = 8 + 64
em2 = [prefix % iring for iring in range(sum_rings, sum_rings + (8 // 2))]

# EM3 list
sum_rings = 8 + 64 + 8
em3 = [prefix % iring for iring in range(sum_rings, sum_rings + (8 // 2))]

# HAD1 list
sum_rings = 8 + 64 + 8 + 8
had1 = [prefix % iring for iring in range(sum_rings, sum_rings + (4 // 2))]

# HAD2 list
sum_rings = 8 + 64 + 8 + 8 + 4
had2 = [prefix % iring for iring in range(sum_rings, sum_rings + (4 // 2))]

# HAD3 list
sum_rings = 8 + 64 + 8 + 8 + 4 + 4
had3 = [prefix % iring for iring in range(sum_rings, sum_rings + (4 // 2))]

col_names = presample + em1 + em2 + em3 + had1 + had2 + had3

In [4]:
# load the data
sgn_df = pd.read_parquet(os.path.join(repo_path, 'data/processed/zee_mb.parquet'))
bkg_df = pd.read_parquet(os.path.join(repo_path, 'data/processed/jf17_mb.parquet'))

m_df = pd.concat([sgn_df, bkg_df], axis=0)


input_cols = [f"cl_ring_{idx}" for idx in range(100)] #col_names #[f"cl_ring_{idx}" for idx in range(100)]

In [5]:
m_df.target.value_counts()

target
1    11984
0    11662
Name: count, dtype: int64

In [6]:
def norm1(data):
    norms = np.abs(data.sum(axis=1))
    norms[norms == 0] = 1
    return data / norms[:, None]

# inputs features normalization
X = norm1(m_df[input_cols].values)
y = m_df.target.values

In [7]:
def build_simple_mlp(input_shape: int, n_layers: int, n_units: int) -> tf.keras.Model:
    """
    Builds a simple Multi-Layer Perceptron (MLP) model.

    Args:
        input_shape (int): The number of input features for the model.
        n_layers (int): The number of hidden layers in the MLP.
        n_units (int): The number of neurons in each hidden layer.

    Returns:
        tf.keras.Model: A compiled Keras model with the specified architecture.
    """
    inputs = tf.keras.layers.Input(shape=(input_shape,), name='Input')
    for ilayer in range(n_layers):
        dense = tf.keras.layers.Dense(n_units, activation='relu', name=f'dense_layer_{ilayer}',
                                      kernel_initializer=tf.keras.initializers.GlorotUniform(seed=512),
                                      bias_initializer='zeros')(inputs if ilayer == 0 else dense)
    # classification layer
    dense = tf.keras.layers.Dense(1, activation='linear', name='output_for_inference',
                                  kernel_initializer=tf.keras.initializers.GlorotUniform(seed=512),
                                  bias_initializer='zeros')(dense) 
    output = tf.keras.layers.Activation('sigmoid', name='output_for_training')(dense)
    model = tf.keras.Model(inputs, output, name="model")
    return model

def class_weight(target: np.ndarray) -> pd.Series:
    """
    Calculate class weights and sample weights for binary classification.

    Args:
        target (np.ndarray): Array of binary target labels (e.g., 0 and 1).

    Returns:
        pd.Series: A pandas Series containing sample weights for each instance, 
                   with the column name 'weight'.
    """
    classes = np.unique(target)
    # [-1,1] or [0,1]
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=target)
    class_weights = {cl: weights[idx] for idx, cl in enumerate(classes)}
    sample_weight = np.ones_like(target, dtype=np.float32)
    sample_weight[target == 1] = weights[1]
    sample_weight[target != 1] = weights[0]
    return pd.Series(sample_weight).to_frame('weight')


In [8]:
model_bce = build_simple_mlp(input_shape=len(input_cols), n_layers=1, n_units=5)
model_geo = build_simple_mlp(input_shape=len(input_cols), n_layers=1, n_units=5)

In [9]:
bce_weights = model_bce.get_weights()
geo_weights = model_geo.get_weights()

for ib, ig in zip(bce_weights, geo_weights):
    print(ib.shape, ig.shape)
    if len(ib.shape) == 1:
        print(np.allclose(ib, ig))
    else:
        print(np.allclose(ib, ig, rtol=1e-5, atol=1e-8))

(100, 5) (100, 5)
True
(5,) (5,)
True
(5, 1) (5, 1)
True
(1,) (1,)
True


In [10]:
patience = 50

In [11]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=512)

loss     = 'binary_crossentropy'
opt      = tf.keras.optimizers.Adam(learning_rate=0.001)

decorators = [Summary(detailed=True)]
# create a training history:
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    if i > 0:
        continue
    x_train, y_train = X[train_index], y[train_index]
    x_test, y_test   = X[test_index] , y[test_index]
    
    
    model_bce.compile(optimizer=opt, 
                  loss=loss, 
                  metrics=['accuracy']
                  )
    
    sp_index_callback = sp_index(validation_data=(x_test, y_test),
                                 patience=patience, verbose=True, save_the_best=True)
    
    bce_history = model_bce.fit(x_train, y_train, epochs=1000,
        batch_size=1024,
        verbose=True,
        validation_data=(x_test, y_test),
        sample_weight=class_weight(y_train),
        callbacks=[sp_index_callback],
        shuffle=True
    ).history
    
    for decorator in decorators:
        decorator(bce_history , {'model':model_bce, 'data':(x_train, y_train),  'data_val':(x_test, y_test) })

Epoch 1/1000
3/3 [==============================] - 0s 3ms/step- loss: 0.6778 - accuracy: 0.54


2025-04-26 01:40:06.768 | INFO     | src.callbacks:on_epoch_end:63 - val_sp: 0.9906 (fa:0.0180, pd:0.9992), patience: 0
2025-04-26 01:40:06.769 | INFO     | src.callbacks:on_epoch_end:68 - save the best configuration here...


21/21 [==============================] - 2s 22ms/step - loss: 0.6738 - accuracy: 0.5980 - val_loss: 0.6664 - val_accuracy: 0.7260 - max_sp_val: 0.9906 - max_sp_fa_val: 0.0180 - max_sp_pd_val: 0.9992 - max_sp_best_epoch_val: 0.0000e+00
Epoch 2/1000
3/3 [==============================] - 0s 3ms/step loss: 0.6654 - accuracy: 0.76


2025-04-26 01:40:06.919 | INFO     | src.callbacks:on_epoch_end:63 - val_sp: 0.9898 (fa:0.0154, pd:0.9950), patience: 0


21/21 [==============================] - 0s 7ms/step - loss: 0.6583 - accuracy: 0.8364 - val_loss: 0.6481 - val_accuracy: 0.9040 - max_sp_val: 0.9898 - max_sp_fa_val: 0.0154 - max_sp_pd_val: 0.9950
Epoch 3/1000
3/3 [==============================] - 0s 2ms/step loss: 0.6493 - accuracy: 0.92


...

2025-04-26 01:40:17.845 | INFO     | src.callbacks:on_epoch_end:63 - val_sp: 0.9992 (fa:0.0009, pd:0.9992), patience: 49


21/21 [==============================] - 0s 7ms/step - loss: 0.0442 - accuracy: 0.9923 - val_loss: 0.0457 - val_accuracy: 0.9899 - max_sp_val: 0.9992 - max_sp_fa_val: 8.5763e-04 - max_sp_pd_val: 0.9992
Epoch 76/1000
3/3 [==============================] - 0s 2ms/step loss: 0.0455 - accuracy: 0.99


2025-04-26 01:40:17.989 | INFO     | src.callbacks:on_epoch_end:63 - val_sp: 0.9992 (fa:0.0009, pd:0.9992), patience: 50
2025-04-26 01:40:17.989 | INFO     | src.callbacks:on_epoch_end:76 - Stopping the Training by SP...


21/21 [==============================] - 0s 7ms/step - loss: 0.0433 - accuracy: 0.9923 - val_loss: 0.0448 - val_accuracy: 0.9899 - max_sp_val: 0.9992 - max_sp_fa_val: 8.5763e-04 - max_sp_pd_val: 0.9992


2025-04-26 01:40:17.991 | INFO     | src.callbacks:on_train_end:84 - Reload the best configuration into the current model...
2025-04-26 01:40:17.997 | INFO     | src.decorators:__call__:74 - Starting the train summary...
2025-04-26 01:40:18.194 | INFO     | src.decorators:__call__:101 - Train samples : Prob. det (0.9994), False Alarm (0.0030), SP (0.9982)
2025-04-26 01:40:18.196 | INFO     | src.decorators:__call__:118 - Validation Samples: Prob. det (0.9992), False Alarm (0.0009), SP (0.9992)
2025-04-26 01:40:18.202 | INFO     | src.decorators:__call__:136 - Operation Samples : Prob. det (0.9994), False Alarm (0.0028), SP (0.9983)


In [15]:
def calc_sp(det, fake):
    return np.sqrt( np.sqrt(det*(1-fake)) * (0.5*(det+(1 - fake))) )

def plot_loss_function(loss, val_loss, best_epoch, axis, loss_name='Binary Cross Entropy'):
    #  'solid', 'dashed', 'dashdot', 'dotted'
    axis.plot(loss,       color='black', ls='dashdot', lw=1.2, label='Training Loss')
    axis.plot(val_loss,   color='red', ls='solid', lw=1.2, label='Test Loss')
    axis.axvline(x=best_epoch, color='black', ls='dotted', lw=1.2, label='Best epoch')
    axis.set_ylabel(loss_name)
    axis.set_xlabel(r'$\#$ epochs')
    axis.legend(loc='best')
    axis.grid()
    axis.set_title(f'{loss_name} Loss Evolution')

    
def plot_det(det, best_epoch, axis):
    #  'solid', 'dashed', 'dashdot', 'dotted'
    axis.plot(det,       color='dodgerblue', ls='dashdot', lw=1.2, label='Test Set')
    axis.axvline(x=best_epoch, color='black', ls='dotted', lw=1.2, label='Best epoch')
    axis.set_ylabel('Detection')
    axis.set_xlabel(r'$\#$ epochs')
    axis.legend(loc='best')
    axis.grid()
    axis.set_title('Detection Evolution (Test set)')

def plot_fake(fake, best_epoch, axis):
    #  'solid', 'dashed', 'dashdot', 'dotted'
    axis.plot(fake,       color='darkred', ls='dashdot', lw=1.2, label='Test Set')
    axis.axvline(x=best_epoch, color='black', ls='dotted', lw=1.2, label='Best epoch')
    axis.set_ylabel('Fake Rate')
    axis.set_xlabel(r'$\#$ epochs')
    axis.legend(loc='best')
    axis.grid()
    axis.set_title('Fake Rate Evolution (Test set)')

def plot_sp(sp_index, best_epoch, axis):
    #  'solid', 'dashed', 'dashdot', 'dotted'
    axis.plot(sp_index,       color='forestgreen', ls='dashdot', lw=1.2, label='Test Set')
    axis.axvline(x=best_epoch, color='black', ls='dotted', lw=1.2, label='Best epoch')
    axis.set_ylabel(r'$SP$ Index')
    axis.set_xlabel(r'$\#$ epochs')
    axis.legend(loc='best')
    axis.grid()
    axis.set_title(r'$SP$ Index Evolution (Test set)')

def plot_roc_curve(roc_fa, roc_pd, reference_dict, axis, ylim=[75, 105], xlim=[-.5, 50]):

    axis.plot(roc_fa*100, roc_pd*100, color='black', linewidth=1.5, ls='dashed')
    axis.set_ylabel('Probability of Dectection $[\%]$')
    axis.set_xlabel('Fake Rate $[\%]$')

    sp_max_index = np.argmax(calc_sp(roc_pd, roc_fa))

    axis.plot(roc_fa[sp_max_index]*100, roc_pd[sp_max_index]*100, '*', 
                 label=r'$SP_{max}$',
                 markersize=10, color=plt.cm.tab10(4))

    axis.set_ylim(ylim)
    axis.set_xlim(xlim)
    axis.grid()
    axis.legend(loc='best')
    axis.set_title('ROC Curve (Test Set)')

def plot_nn_histogram(hist_dict, reference_dict, axis, threshold=None):
        axis.hist(hist_dict['val_bkg'][1][:-1], hist_dict['val_bkg'][1], weights=hist_dict['val_bkg'][0], histtype='bar', 
                     color='slategray', lw=1.5, 
                     label='jets', alpha=.5)
        axis.hist(hist_dict['val_sgn'][1][:-1], hist_dict['val_sgn'][1], weights=hist_dict['val_sgn'][0], histtype='bar', 
                     color='blue', lw=1.5, 
                     label='electron', alpha=.5)

        if threshold is not None:
            axis.axvline(x=threshold, ls='solid',
                         label=r'$SP_{max}$ Threshold', color='black', lw=1.5)
        colors = iter([plt.cm.tab10(i) for i in range(4)])

        axis.set_title('NN Output Histogram (Test Set)')
        axis.legend(loc='best')
        axis.grid()
        axis.set_yscale('log')
        axis.set_ylabel('Counts')
        axis.set_xlabel('NN Output')

def plot_training_resume(history, output_path, train_tag='Electron-Pion', roc_xlim=[-.5, 35], roc_ylim=[75, 105], loss_name='Binary Cross Entropy'):
    os.makedirs(os.path.join(output_path, f"{train_tag}"), exist_ok=True)
    o_path = os.path.join(output_path, f"{train_tag}")
    #print(o_path)
    # let's moving per sort
    fig, ax  = plt.subplots(nrows=2, ncols=3, sharex=False, sharey=False, figsize=(20,10))
    axes     = ax.flatten()
    
    hist_dict        = history['summary']['hists']
    thr_max_sap      = history['summary']['threshold_val']
    roc_pd, roc_fa   = history['summary']['rocs']['roc_val']

    best_epoch       = history['max_sp_best_epoch_val'][-1]
    loss             = history['loss']
    val_loss         = history['val_loss']

    sp_index         = history['max_sp_val']
    det              = history['max_sp_pd_val']
    fake             = history['max_sp_fa_val']


    # plot loss
    plot_loss_function(loss, val_loss, best_epoch, axis=axes[0], loss_name=loss_name)
    # plot roc curve
    plot_roc_curve(roc_fa=roc_fa, roc_pd=roc_pd, reference_dict={}, 
                    axis=axes[1], xlim=roc_xlim, ylim=roc_ylim)
    # plot nn output 
    plot_nn_histogram(hist_dict, reference_dict={}, axis=axes[2], threshold=thr_max_sap)
    # plot the detection
    plot_det(det, best_epoch=best_epoch, axis=axes[3])
    # plot sp index
    plot_sp(sp_index=sp_index, best_epoch=best_epoch, axis=axes[4])
    # plot fake
    plot_fake(fake=fake, best_epoch=best_epoch, axis=axes[5])

    fig.suptitle(r'Train Resume', fontsize=15)
        
    plt.savefig(os.path.join(o_path, 'train_resume.png' ), dpi=150, facecolor='white', bbox_inches='tight')
    fig.clf()
    plt.close()

In [13]:
plot_training_resume(bce_history, output_path=repo_path+'/notebooks', train_tag='zee-jf17-mb', roc_xlim=[-.5, 50], roc_ylim=[0, 105])

# geodesic

In [14]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=512)

def geodesic_loss(y_true, y_pred):
    """
    Geodesic loss function for the neural network.
    """
    # Calculate the geodesic connection
    bce            = tf.keras.losses.BinaryCrossentropy(reduction='none')
    conection      = bce(y_true, y_pred)
    #phi_t          = tf.reduce_sum(conection, axis=-1, keepdims=True)
    #print(conection)
    #print(phi_t)
    exp_connection = tf.math.exp(conection)
    #print(exp_connection)
    
    # Return the sum of the geodesic connection
    return tf.reduce_mean(exp_connection, axis=-1, keepdims=True) - 1

loss     = geodesic_loss
opt      = tf.keras.optimizers.Adam(learning_rate=0.001)


decorators = [Summary(detailed=True)]
# create a training history:
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    if i > 0:
        continue
    x_train, y_train = X[train_index], y[train_index]
    x_test, y_test   = X[test_index] , y[test_index]
    
    
    model_geo.compile(optimizer=opt, 
                  loss=loss, 
                  metrics=['accuracy']
                  )
    
    sp_index_callback = sp_index(validation_data=(x_test, y_test),
                                 patience=patience, verbose=True, save_the_best=True)
    
    geo_history = model_geo.fit(x_train, y_train, epochs=1000,
        batch_size=1024,
        verbose=True,
        validation_data=(x_test, y_test),
        sample_weight=class_weight(y_train),
        callbacks=[sp_index_callback],
        shuffle=True
    ).history
    
    for decorator in decorators:
        decorator(geo_history , {'model':model_geo, 'data':(x_train, y_train),  'data_val':(x_test, y_test) })

Epoch 1/1000
3/3 [==============================] - 0s 3ms/step- loss: 0.9731 - accuracy: 0.52


2025-04-26 01:42:10.832 | INFO     | src.callbacks:on_epoch_end:63 - val_sp: 0.9906 (fa:0.0163, pd:0.9975), patience: 0
2025-04-26 01:42:10.833 | INFO     | src.callbacks:on_epoch_end:68 - save the best configuration here...


21/21 [==============================] - 1s 18ms/step - loss: 0.9613 - accuracy: 0.6430 - val_loss: 0.9475 - val_accuracy: 0.7848 - max_sp_val: 0.9906 - max_sp_fa_val: 0.0163 - max_sp_pd_val: 0.9975 - max_sp_best_epoch_val: 0.0000e+00
Epoch 2/1000
3/3 [==============================] - 0s 2ms/step loss: 0.9455 - accuracy: 0.81


2025-04-26 01:42:10.983 | INFO     | src.callbacks:on_epoch_end:63 - val_sp: 0.9898 (fa:0.0154, pd:0.9950), patience: 0


21/21 [==============================] - 0s 7ms/step - loss: 0.9312 - accuracy: 0.8746 - val_loss: 0.9124 - val_accuracy: 0.9277 - max_sp_val: 0.9898 - max_sp_fa_val: 0.0154 - max_sp_pd_val: 0.9950
Epoch 3/1000
3/3 [==============================] - 0s 2ms/step loss: 0.9095 - accuracy: 0.93


...

2025-04-26 01:42:21.767 | INFO     | src.callbacks:on_epoch_end:63 - val_sp: 0.9992 (fa:0.0009, pd:0.9992), patience: 49


21/21 [==============================] - 0s 7ms/step - loss: 0.0758 - accuracy: 0.9942 - val_loss: 0.0737 - val_accuracy: 0.9924 - max_sp_val: 0.9992 - max_sp_fa_val: 8.5763e-04 - max_sp_pd_val: 0.9992
Epoch 76/1000
3/3 [==============================] - 0s 2ms/step loss: 0.0612 - accuracy: 0.99


2025-04-26 01:42:21.914 | INFO     | src.callbacks:on_epoch_end:63 - val_sp: 0.9992 (fa:0.0009, pd:0.9992), patience: 50
2025-04-26 01:42:21.915 | INFO     | src.callbacks:on_epoch_end:76 - Stopping the Training by SP...


21/21 [==============================] - 0s 7ms/step - loss: 0.0747 - accuracy: 0.9942 - val_loss: 0.0725 - val_accuracy: 0.9924 - max_sp_val: 0.9992 - max_sp_fa_val: 8.5763e-04 - max_sp_pd_val: 0.9992


2025-04-26 01:42:21.918 | INFO     | src.callbacks:on_train_end:84 - Reload the best configuration into the current model...
2025-04-26 01:42:21.922 | INFO     | src.decorators:__call__:74 - Starting the train summary...
2025-04-26 01:42:22.100 | INFO     | src.decorators:__call__:101 - Train samples : Prob. det (0.9995), False Alarm (0.0030), SP (0.9982)
2025-04-26 01:42:22.102 | INFO     | src.decorators:__call__:118 - Validation Samples: Prob. det (0.9992), False Alarm (0.0009), SP (0.9992)
2025-04-26 01:42:22.107 | INFO     | src.decorators:__call__:136 - Operation Samples : Prob. det (0.9995), False Alarm (0.0029), SP (0.9983)


In [16]:
plot_training_resume(geo_history, output_path=repo_path+'/notebooks', train_tag='zee-jf17-mb-geodesic',
                     roc_xlim=[-.5, 50], roc_ylim=[0, 105], loss_name='Geodesic Connection')

In [17]:
bce_history['summary']['threshold_op'], geo_history['summary']['threshold_op']

(0.72332805, 0.68657744)

In [18]:
bce_history['summary']['max_sp_val'], geo_history['summary']['max_sp_val']

(0.9991541693201127, 0.9991541693201127)